<div style="background-color:green; color:white; padding:10px; font-size:20px">
1 - Setup

In [1]:
!pip install boto3

  Obtaining dependency information for urllib3<1.27,>=1.25.4 from https://files.pythonhosted.org/packages/c5/05/c214b32d21c0b465506f95c4f28ccbcba15022e000b043b72b3df7728471/urllib3-1.26.16-py2.py3-none-any.whl.metadata
  Using cached urllib3-1.26.16-py2.py3-none-any.whl.metadata (48 kB)
Using cached urllib3-1.26.16-py2.py3-none-any.whl (143 kB)
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.4
    Uninstalling urllib3-2.0.4:
      Successfully uninstalled urllib3-2.0.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the followi

In [2]:
import random
import time
import boto3
import pandas as pd

# Project Scripts
import data_handling

<div style="background-color:teal; color:white; padding:0px; font-size:20px">
♺ Refresh

In [3]:
# Re-freshing the module import for development purposes
from importlib import reload
reload(data_handling)

<module 'data_handling' from '/root/projects/behavior_project/data_handling.py'>

#### Sagemaker User & IAM Credentials

In [4]:
# Create credentials session (log-in to AWS account)
sess = boto3.Session()
# Get information about current User
sts = sess.client("sts")
response = sts.get_caller_identity()

print("Your AWS Account:  717145514721")
print("Logged-in Account:", response["Account"])
print("User ARN:", response["Arn"])

Your AWS Account:  717145514721
Logged-in Account: 717145514721
User ARN: arn:aws:sts::717145514721:assumed-role/AWS-SageMakerFullAccess-Role/SageMaker


### S3 Connection

In [5]:
import boto3

# Create a session using your AWS credentials
s3 = boto3.client("s3")

# List all bucket names
response = s3.list_buckets()

# Get a list of all bucket names from the response
buckets = [bucket["Name"] for bucket in response["Buckets"]]

# Print out the bucket list
print("Bucket List: %s" % buckets)

Bucket List: ['behavior-project', 'sagemaker-us-east-1-717145514721']


/opt/conda/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


<div style="background-color:green; color:white; padding:10px; font-size:20px">
2 - Data Import

In [6]:
from io import StringIO

from data_handling import load_to_dict, view_rand_df

In [7]:
mPFC_session_df_path = "s3://behavior-project/processed-data/processed-data/session-df-dictionaries/mpfc-icss//"
FR_session_df_path = "s3://behavior-project/processed-data/processed-data/session-df-dictionaries/fr-icss//"

dtype_conversion_dict = {"float64": "float32", "int64": "bool"}

mPFC_session_df_dict = load_to_dict(
    s3_url=mPFC_session_df_path, dtype_dict=dtype_conversion_dict
)

FR_session_df_dict = load_to_dict(
    s3_url=FR_session_df_path, dtype_dict=dtype_conversion_dict
)

In [8]:
print(next(iter(mPFC_session_df_dict.keys())))
next(iter(mPFC_session_df_dict.values())).head(3)

C58_R10_ChR2_mPFC_30Hz_S01_P02_N_NON_NON


,time,centre_position_x,centre_position_y,in_t,in_d,in_h,in_v,session_zone_(v)
0,0.000,NaN,NaN,False,False,True,False,False
1,1.278,475.0,324.0,False,False,True,False,False
2,1.285,475.0,324.0,False,False,True,False,False


In [9]:
print(next(iter(FR_session_df_dict.keys())))
next(iter(FR_session_df_dict.values())).head(3)

C1000_C42A_ChR2_xPVT_F00_S01_P05_N_NON_NON


,time,centre_position_x,centre_position_y,in_t,in_d,in_h,in_v,session_zone_(t)
0,0.000,NaN,NaN,False,False,False,False,False
1,0.287,271.0,260.0,False,False,False,False,False
2,0.407,261.0,267.0,False,False,False,False,False


In [10]:
# Specify the S3 bucket and the file key
bucket_name = "behavior-project"
file_key = "processed-data/processed-data/session-info-dfs/mPFC_session_info_df.csv"

# Get the CSV file from S3
csv_obj = s3.get_object(Bucket=bucket_name, Key=file_key)
body = csv_obj["Body"].read().decode("utf-8")

# Convert the CSV data to a pandas DataFrame
mPFC_session_info_df = pd.read_csv(StringIO(body))
mPFC_session_info_df.head(3)

,session_id,animal_id_1,animal_id_2,virus,stim_target,stim_freq,session_num,pod_num,stim_zone,sens_treat_1,sens_treat_2,stim_session_list
0,C58_R10_ChR2_mPFC_30Hz_S01_P02_N_NON_NON,C58,R10,ChR2,mPFC,30Hz,S01,P02,N,NON,NON,NVN
1,C58_R10_ChR2_mPFC_30Hz_S02_P02_V_NON_NON,C58,R10,ChR2,mPFC,30Hz,S02,P02,V,NON,NON,NVN
2,C58_R10_ChR2_mPFC_30Hz_S03_P02_N_NON_NON,C58,R10,ChR2,mPFC,30Hz,S03,P02,N,NON,NON,NVN


In [11]:
# Specify the S3 bucket and the file key
bucket_name = "behavior-project"
file_key = "processed-data/processed-data/session-info-dfs/FR_session_info_df.csv"

# Get the CSV file from S3
csv_obj = s3.get_object(Bucket=bucket_name, Key=file_key)
body = csv_obj["Body"].read().decode("utf-8")

# Convert the CSV data to a pandas DataFrame
FR_session_info_df = pd.read_csv(StringIO(body))
FR_session_info_df.head(3)

,session_id,animal_id_1,animal_id_2,virus,stim_target,stim_freq,session_num,pod_num,stim_zone,sens_treat_1,sens_treat_2,stim_session_list
0,C1000_C42A_ChR2_xPVT_F00_S01_P05_N_NON_NON,C1000,C42A,ChR2,xPVT,F00,S01,P05,N,NON,NON,NTD
1,C1000_C42A_ChR2_xPVT_F20_S02_P05_T_NON_NON,C1000,C42A,ChR2,xPVT,F20,S02,P05,T,NON,NON,NTD
2,C1000_C42A_ChR2_xPVT_F20_S03_P05_D_NON_NON,C1000,C42A,ChR2,xPVT,F20,S03,P05,D,NON,NON,NTD


<div style="background-color:green; color:white; padding:10px; font-size:20px">
3 - Trace Creation

## Function Overview

<img src="./images/Trace_Creation_Function.png" width="400" />

Traces are essentially created by 'cutting' the datastream at stim-zone status transitions and then stepping (at most) 5 seconds backwards in time to include the behavior leading up to the zone entry. The resulting traces are shorter if the animal enters the zone again before the 5 seconds had transpired.

In [12]:
from data_handling import interpolate_trace, trim_to_5, zone_entry

In [13]:
DICT = FR_session_df_dict
INFO_DF = FR_session_info_df
STIM_COL_IDX = 7
TIME_COL_IDX = 0

untrimmed_traces = zone_entry(df_dict = DICT,
                              info_df = INFO_DF,
                              stim_col_idx = STIM_COL_IDX)

FR_trace_dict = trim_to_5(dict_list_df = untrimmed_traces,
                       stim_col_idx = STIM_COL_IDX,
                       time_col_idx = TIME_COL_IDX)

#### Trace Time Durations Check

In [14]:
from datetime import timedelta

TRACE_DICT = FR_trace_dict

problem_found = False

for key, value in TRACE_DICT.items():

    for index, df in enumerate(value):
        time_delta = df["time"].max() - df["time"].min()
        five_sec = timedelta(seconds=5)
        zero = timedelta(seconds=0)

        if time_delta > five_sec:
            print(
                "trace too long! - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - "
            )
            print(f"{key}, trace {index} is too long.")
            problem_found = True
        elif df.empty:
            print(f"{key}, trace {index} is empty.")
            problem_found = True
        elif time_delta == zero:
            print(f"{key}, trace {index} has no time.")
            problem_found = True
            print(len(df))

if not problem_found:
    print("No Trace Duration Problems Found")

No Trace Duration Problems Found


<div style="background-color:green; color:white; padding:10px; font-size:20px">
4 - Trace Plotting

<div style="background-color:darkseagreen; color:black; padding:0px; font-size:20px">
Full Dataset Plot and Save

In [15]:
import itertools
from io import BytesIO

import seaborn as sns
from matplotlib import cm
from matplotlib import pyplot as plt

<div style="background-color:teal; color:white; padding:0px; font-size:20px">
mPFC

#### Remove Unwanted Columns From Trace Dict

#### Save Files

<div style="background-color:teal; color:white; padding:0px; font-size:20px">
FR

In [16]:
TRACE_DICT = FR_trace_dict
CENTERX_COL_IDX = 1
CENTERY_COL_IDX = 2
TIME_COL_IDX = 0

S3_DIR_PATH = "s3://behavior-project/processed-data/trace-images/fr-icss"

# Split the s3_path into bucket_name and prefix
bucket_name, prefix = S3_DIR_PATH.replace("s3://", "").split("/", 1)

# Column List
session_id_col = []
trace_num_col = []
start_time_col = []
end_time_col = []
s3_url_col = []

# Fig Axes Parameters
xmin, xmax = 0, 550
ymin, ymax = 0, 550

# SESSION LEVEL - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# for name, tracelist in itertools.islice(TRACE_DICT.items(), 1): # For subsection testing of the loop

for name, tracelist in TRACE_DICT.items():

    # TRACE LEVEL - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    for list_idx, trace in enumerate(tracelist):

        # Interpolate between datapoints for plotting
        trace_interp = interpolate_trace(
            trace,
            centerX_col_idx=CENTERX_COL_IDX,
            centerY_col_idx=CENTERY_COL_IDX,
            time_col_idx=TIME_COL_IDX,
        )

        # Instantiate Figure Size & params
        dpi = 128
        file_extension = "png"
        fig, ax = plt.subplots(figsize=(224 / dpi, 224 / dpi), dpi=128)

        # Plot
        sns.scatterplot(
            data=trace_interp,
            x=trace_interp.iloc[:, 0],
            y=trace_interp.iloc[:, 1],
            hue="Time_d",
            hue_norm=(0, 5),
            edgecolor="none",
            legend=False,
            alpha=0.3,
            ax=ax,
        )  # Adding this to close the figure later in the loop

        # Setting Axes Range
        plt.xlim(xmin, xmax)
        plt.ylim(ymin, ymax)
        plt.axis("off")  # Turn axes marks off

        # Define file_name
        file_name = f"{prefix}/{name}_{list_idx}.{file_extension}"

        # Load Dataframe Information
        session_id_col.append(name)
        trace_num_col.append(f"{list_idx}")
        trace[TIME_COL_IDX] = pd.to_datetime(
            trace.columns[TIME_COL_IDX], errors="coerce"
        )  # *** Converting col to datetime
        start_time_col.append(trace[trace.columns[TIME_COL_IDX]].min())
        end_time_col.append(trace[trace.columns[TIME_COL_IDX]].max())
        s3_url_col.append(f"{bucket_name}/{file_name}")

        # Save the plot to a BytesIO object
        img_data = BytesIO()
        plt.savefig(img_data, dpi=128, format="png", pad_inches=0)
        img_data.seek(0)

        # Save the plot to the S3 bucket
        s3.upload_fileobj(img_data, Bucket=bucket_name, Key=file_name)

        # Close the figure to free up memory
        plt.close(fig)

# MASTER DF CREATION - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# Temp dict. to load column data
df_build_dict = {
    "session_id": session_id_col,
    "trace_num": trace_num_col,
    "start_time": start_time_col,
    "end_time": end_time_col,
    "s3_url": s3_url_col,
}

In [20]:
# Build df from dict
FR_trace_info = pd.DataFrame(df_build_dict)
FR_trace_info.head(3)

,session_id,trace_num,start_time,end_time,s3_url
0,C1000_C42A_ChR2_xPVT_F00_S01_P05_N_NON_NON,0,1970-01-01 00:00:45.854000091,1970-01-01 00:00:50.800998687,behavior-project/processed-data/trace-images/f...
1,C1000_C42A_ChR2_xPVT_F00_S01_P05_N_NON_NON,1,1970-01-01 00:01:03.058998107,1970-01-01 00:01:08.055000305,behavior-project/processed-data/trace-images/f...
2,C1000_C42A_ChR2_xPVT_F00_S01_P05_N_NON_NON,2,1970-01-01 00:01:33.384002685,1970-01-01 00:01:38.366996765,behavior-project/processed-data/trace-images/f...


#### Remove Unwanted Columns From Trace Dict

In [21]:
from data_handling import remove_columns_from_dict

drop_col_list = ['Time_d', 0]

# Call the function to remove the columns
FR_trace_dict = remove_columns_from_dict(FR_trace_dict, drop_col_list)

In [23]:
next(iter(FR_trace_dict.values()))[0].head(3)

,time,centre_position_x,centre_position_y,in_t,in_d,in_h,in_v,session_zone_(t),trace_num,session_id
389,1970-01-01 00:00:45.854000091,448.0,383.0,False,False,True,False,False,0,C1000_C42A_ChR2_xPVT_F00_S01_P05_N_NON_NON
390,1970-01-01 00:00:45.963001251,448.0,382.0,False,False,True,False,False,0,C1000_C42A_ChR2_xPVT_F00_S01_P05_N_NON_NON
391,1970-01-01 00:00:46.076000213,448.0,382.0,False,False,True,False,False,0,C1000_C42A_ChR2_xPVT_F00_S01_P05_N_NON_NON


#### Save Files

In [24]:
# mpfc_trace_dict:

FILE = FR_trace_dict
S3_DIR_PATH = "s3://behavior-project/processed-data/processed-data/trace-df-dictionaries/fr-icss"

# Split the s3_path into bucket_name and prefix
bucket_name, prefix = S3_DIR_PATH.replace("s3://", "").split("/", 1)

# Iterate through the keys and lists of DataFrames in the dictionary
for key, df_list in FILE.items():
    # Iterate through the DataFrames in the list, keeping track of the index
    for list_index, df in enumerate(df_list):
        # Create a CSV buffer
        csv_buffer = StringIO()

        # Convert the dataframe to CSV and write it to the buffer
        df.to_csv(csv_buffer, index=False)

        # Create a file name based on the dictionary key and list index
        file_name = f"{prefix}/{key}_{list_index}.csv"

        # Upload the CSV file to S3
        s3.put_object(Bucket=bucket_name, Key=file_name, Body=csv_buffer.getvalue())

In [25]:
# mpfc_trace_info

FILE = FR_trace_info
S3_DIR_PATH = "s3://behavior-project/processed-data/processed-data/trace-info-dfs"
FILE_NAME = 'fr-trace-info-df.csv'

# Split the s3_path into bucket_name and prefix
bucket_name, prefix = S3_DIR_PATH.replace("s3://", "").split("/", 1)
file_path = f"{prefix}/{FILE_NAME}"

# Create a CSV buffer
csv_buffer = StringIO()

# Convert the dataframe to CSV and write it to the buffer
FILE.to_csv(csv_buffer, index=False)

# Upload the CSV file to S3
s3.put_object(Bucket=bucket_name, Key=file_path, Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'JZ4JRQ05KNEJ9TT6',
  'HostId': '8jmP4rTsebAo/tHVvlFCQethr4O9XvVsfH/n4LKOb3dGnSzIxtxwr4w2JdQX0RyYcw5ZJ+ykbCI=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '8jmP4rTsebAo/tHVvlFCQethr4O9XvVsfH/n4LKOb3dGnSzIxtxwr4w2JdQX0RyYcw5ZJ+ykbCI=',
   'x-amz-request-id': 'JZ4JRQ05KNEJ9TT6',
   'date': 'Wed, 02 Aug 2023 22:00:36 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"53080dcd0114f1516c93914146a47e82"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"53080dcd0114f1516c93914146a47e82"',
 'ServerSideEncryption': 'AES256'}